In [1]:
!pip install pyxdameraulevenshtein tensorflow editdistance

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 62 kB 8.2 MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 479.6 MB 81.9 MB/s eta 0:00:01     |████████████▊                   | 190.3 MB 13.9 MB/s eta 0:00:21        | 204.5 MB 67.6 MB/s eta 0:00:05     |████████████████▋               | 248.9 MB 67.6 MB/s eta 0:00:04
     |████████████████████████████████| 402 kB 48.5 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 63.7 MB/s eta 0:00:01
     |████████████████████████████████| 24.5 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 80.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 49.4 MB/s eta 0:00:01


In [3]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import json
import tensorflow as tf
# import coremltools as ct
import numpy as np
import re
import pandas as pd
import random
from numpy import argmax
import editdistance
from tqdm import tqdm
from itertools import groupby
from itertools import islice
from tqdm import tqdm

2025-05-05 09:19:21.664599: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-05 09:19:22.516007: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-05-05 09:19:22.516212: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

In [2]:
from pyxdameraulevenshtein import damerau_levenshtein_distance

In [3]:
def sort_acc_edit_distance(source, output):
    words = []
    dists = []
    probs = []
    
    for word, prob in output:
        dist = editdistance.eval(source, word[:len(source)])
        words.append(word)
        dists.append(dist)
        probs.append(prob)
    
    if len(words) >= 2 and words[0].startswith(words[1]):
        if len(probs) > 1 and (probs[1] + (probs[0] / 2)) > probs[0]:
            if len(words[1]) > 1:
                # Return the sorted list with adjusted probabilities
                return [(words[1], probs[1] + (probs[0]) / 2)] + list(zip(words, probs))[1:]
   
    return list(zip(words, probs))[:3]

In [4]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class LRUTrie:
    def __init__(self, max_size):
        self.root = TrieNode()
        self.max_size = max_size

    def insert(self, word, frequency=1):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
        node.frequency = frequency

    def search(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]
        if(node.is_end_of_word):
            node.frequency+=1
            return True
        else:
            return False

    def starts_with(self, prefix):
#        start=time.time()
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []  # No words start with the given prefix
            node = node.children[char]

        # Gather all words starting from this prefix node
        words = []
        self._collect_words(node, prefix, words)
#        print("starts_with time",time.time()-start)
        return words

    def _collect_words(self, node, prefix, words): #getting all the words for a given prefix
        if node.is_end_of_word:
            words.append((prefix, node.frequency))
        
        for char, next_node in node.children.items():
            self._collect_words(next_node, prefix + char, words)
            
    def starts_with_new(self, prefix, min_length=None, max_length=None):
    # Locate the node corresponding to the prefix
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []  # No words start with the given prefix
            node = node.children[char]

        # Gather all words starting from this prefix node
        words = []
        self._collect_words_new(node, prefix, words, min_length, max_length)
        return words

    def _collect_words_new(self, node, prefix, words, min_length=None, max_length=None):
        # Check if the current prefix is too short or too long
        if min_length is not None and len(prefix) < min_length:
            pass  # Continue traversal to allow the prefix to grow longer
        elif max_length is not None and len(prefix) > max_length:
            return  # Skip this branch of the trie

        if node.is_end_of_word and (min_length is None or len(prefix) >= min_length):
            words.append((prefix, node.frequency))

        for char, next_node in node.children.items():
            self._collect_words_new(next_node, prefix + char, words, min_length, max_length)
    def calculate_size(self):
        return self._count_nodes(self.root)

    def _count_nodes(self, node):
        count = 1  # Count the current node
        for child in node.children.values():
            count += self._count_nodes(child)
        return count



uni_trie = LRUTrie(max_size=500000)
bi_trie = LRUTrie(max_size=500000)
#tri_trie = LRUTrie(max_size=1000)

In [5]:
df=pd.read_csv("en-ml-bigrams.csv")
df.dropna(inplace=True)
df2=pd.read_csv("en-ta-bigrams.csv")
df2.dropna(inplace=True)
df3=pd.read_csv("en-te-bigrams.csv")
df3.dropna(inplace=True)

In [6]:
df.shape,df2.shape,df3.shape

((344692, 3), (351550, 3), (337955, 3))

In [7]:
df3

,0,1,2
0,aab,nai,9
1,aab,tu,10
2,aab,nhi,11
3,aab,aap,12
4,aab,se,13
...,...,...,...
338071,zyada,nhi,13
338072,zyada,hi,14
338073,zyada,hai,15
338074,zygote,of,15


In [8]:
u=[]
b=[]
for i in tqdm(df.iterrows()):
    i=i[1]
    temp_word=[i["0"],i["1"]]
    temp_word=temp_word*i["2"]
    for j in temp_word:
        u.append(j)
    for j in [[i["0"],i["1"]]]*i["2"]:
        b.append(j)
for i in tqdm(df2.iterrows()):
    i=i[1]
    temp_word=[i["0"],i["1"]]
    temp_word=temp_word*i["2"]
    for j in temp_word:
        u.append(j)
    for j in [[i["0"],i["1"]]]*i["2"]:
        b.append(j)
for i in tqdm(df3.iterrows()):
    i=i[1]
    temp_word=[i["0"],i["1"]]
    temp_word=temp_word*i["2"]
    for j in temp_word:
        u.append(j)
    for j in [i["0"],i["1"]]*i["2"]:
        b.append(j)

344692it [00:15, 22210.89it/s]
351550it [00:15, 22567.28it/s]
337955it [00:14, 22601.33it/s]


In [9]:
unique_words=[]
df = pd.read_csv('../../../../Achyut_work/Trie_based_suggestions/resources/en-hi_normalised_unigrams.csv')
unigrams = list(df['0'].values)
frequencies = list(df['1'].values)
words_with_frequencies = [(unigrams[i], frequencies[i]) for i in range(len(unigrams))]
for word, freq in tqdm(words_with_frequencies):
    if(freq>0):
        uni_trie.insert(word)
        freq-=1
        for j in range(freq):
            uni_trie.search(word)
for word, freq in tqdm(words_with_frequencies):
    unique_words.append(word)

100%|██████████████████████████████████████████████████████████████████████| 138296/138296 [00:00<00:00, 3521826.96it/s]


In [10]:
for word in tqdm(u):
    if(uni_trie.search(word)):
        continue
    uni_trie.insert(word)
            
for word in tqdm(u):
    unique_words.append(word)

for word in tqdm(b):
    if(bi_trie.search(word)):
        continue
    bi_trie.insert(word)
            
for word in tqdm(b):
    unique_words.append(word)

100%|██████████████████████████████████████████████████████████████████| 15698930/15698930 [00:03<00:00, 4853901.05it/s]


In [11]:
unique_words=[]
df = pd.read_csv('Bigrams/Bi_gram.csv')
unigrams = list(df['sentence'].values)
frequencies = list(df['frequency'].values)
words_with_frequencies = [(unigrams[i], frequencies[i]) for i in range(len(unigrams))]
for word, freq in tqdm(words_with_frequencies):
    if(freq>0):
        bi_trie.insert(word)
        freq-=1
        for j in range(freq):
            bi_trie.search(word)
for word, freq in tqdm(words_with_frequencies):
    unique_words.append(word)
unique_words=list(set(unique_words))

100%|██████████████████████████████████████████████████████████████████████| 345477/345477 [00:00<00:00, 3348288.72it/s]


TypeError: unhashable type: 'list'

In [12]:
unique_words

['mujhe',
 'sorry',
 'hello',
 'khana',
 'morning',
 'achha',
 'happy',
 'bahut',
 'accha',
 'phone',
 'video',
 'night',
 'kaise',
 'karna',
 'birthday',
 'kabhi',
 'nahi',
 'aapko',
 'kuch',
 'batao',
 'gussa',
 'photo',
 'bhaiya',
 'please',
 'jaldi',
 'karne',
 'karta',
 'kuchh',
 'bhai',
 'abhi',
 'acha',
 'baat',
 'thik',
 'call',
 'mummy',
 'bolo',
 'isliye',
 'maine',
 'pagal',
 'online',
 'lagta',
 'lekin',
 'karti',
 'karte',
 'mere',
 'good',
 'number',
 'thoda',
 'chahiye',
 'jyada',
 'pahle',
 'bolna',
 'dekha',
 'tumko',
 'dekho',
 'kitna',
 'thodi',
 'love',
 'thanks',
 'thank',
 'tumhe',
 'reply',
 'whatsapp',
 'jayega',
 'meri',
 'tujhe',
 'khaya',
 'block',
 'tumhare',
 'kiya',
 'mera',
 'liye',
 'shadi',
 'ladki',
 'chalo',
 'sakta',
 'mobile',
 'kharab',
 'nhi',
 'kaha',
 'jaan',
 'raha',
 'theek',
 'friend',
 'aapke',
 'status',
 'dunga',
 'nikal',
 'nahin',
 'tumne',
 'chala',
 'hai',
 'hindi',
 'kitne',
 'bhabhi',
 'aapka',
 'aapne',
 'kasam',
 'bhejo',
 'sakte',

In [13]:
#uni_trie.size(),bi_trie.size()

In [14]:
#uni_trie.save_to_file("uni_trie.txt")
#bi_trie.save_to_file("bi_trie.txt")


In [15]:
#updated
def suggestions_result(word,suggestions,unigram, code, output):#add frequency and input word
    if suggestions:
        suggestions = sorted(suggestions, key=lambda x: x[1], reverse=True)[:20]
        for i in range(len(suggestions)):
            suggestions[i][1] = calculate_confidence(word, suggestions[i][0], suggestions[i][1])
        #return suggestions
    # Fallback to closest word matching only if 'uni' mode is specified
    #if code != "uni":
        #return []
    if(code=="bi"):
        suggestions = [[word, min(threshold + 0.3, 1)] if word in output and threshold > 0.4 and output[word] > 0.4
    else [word, min(threshold + 0.15, 1)]
    for word, threshold in suggestions]
    suggestions+=closest_word(word, unigram)
    return suggestions

def calculate_confidence(input_word, candidate_word, frequency, max_frequency_in_dataset=350):
    if len(input_word) > 5:
        SIMILARITY_WEIGHT = 0.3
        FREQUENCY_WEIGHT = 0.7
    else:
        SIMILARITY_WEIGHT = 0.2
        FREQUENCY_WEIGHT = 0.8 
    LENGTH_MISMATCH_PENALTY = 0.1  # Adjusted to reduce excessive penalties
    # Damerau-Levenshtein Distance (ensure you use an optimized implementation)
    edit_distance = damerau_levenshtein_distance(input_word, candidate_word)

    # Normalized edit similarity (Higher similarity = closer to 1)
    max_len = max(len(input_word), len(candidate_word))
    edit_similarity = 1 - (edit_distance / max_len)

    # Length mismatch penalty (scaled relative to word length)
    length_difference = abs(len(input_word) - len(candidate_word))
    length_penalty = LENGTH_MISMATCH_PENALTY * (length_difference / max_len)

    # Frequency normalization with logarithmic scaling
    normalized_frequency = math.log1p(frequency)  # log1p(x) = log(1 + x) for numerical stability
    max_normalized_frequency = math.log1p(max_frequency_in_dataset)
    frequency_score = normalized_frequency / max_normalized_frequency

    # Confidence calculation
    confidence = (
        (SIMILARITY_WEIGHT * edit_similarity) +
        (FREQUENCY_WEIGHT * frequency_score)
    )
    confidence -= length_penalty  # Subtract penalty for length mismatch

    # Handle perfect match edge case
    if input_word == candidate_word:
        return 1.0

    # Ensure confidence is bounded between 0 and 1
    return max(0, min(confidence, 1))


def closest_word(word, unigram):
    min_len = len(word) - 1
    max_len = len(word) + 2
    prefix = word[:3] 
    all_words = unigram.starts_with_new(prefix)
    all_words = [w for w in all_words if min_len <= len(w[0]) <= max_len]
    all_words.sort(key=lambda x: x[1], reverse=True)
    closest_words = []
    min_distance = float('inf')
    max_distance = 3  

    for candidate_word, frequency in all_words:
        distance = damerau_levenshtein_distance(word, candidate_word)

        # Skip words that are to distant
        if distance > max_distance:
            continue
        
        # Update closest words if a new minimum distance is found
        if distance < min_distance:
            min_distance = distance
            closest_words = [(candidate_word, frequency)]
        elif distance == min_distance:
            closest_words.append((candidate_word, frequency))

    # Sort closest words by frequency and get top 3
    closest_words.sort(key=lambda x: x[1], reverse=True)
    top_closest_words = closest_words[:3]

    # Calculate confidence scores for each of the top closest words
    top_closest_words = [[w, calculate_confidence(word, w, freq)] for w, freq in top_closest_words]
    return top_closest_words


def update_output(trie, ngram, key, output, code, multiplier=1):
    temp_results = trie.starts_with(ngram)
    temp_results = [[result.split(" ")[-1],rest[0]] for result, *rest in temp_results]
    result = suggestions_result(key.split(" ")[-1], temp_results, uni_trie, code, output)
    output.update({item[0]: item[1] * multiplier for item in result})
    
def predict(trie, ngram, output, multiplier=1):
    temp_results = trie.starts_with(ngram)
    suggestions = [[result.split(" ")[-1],rest[0]] for result, *rest in temp_results]
    if suggestions:
        suggestions = sorted(suggestions, key=lambda x: x[0], reverse=True)[:3]
    output.update({item[0]: item[1] * multiplier for item in suggestions})


def softmax_normalize_dict(scores):
    if not scores:
        return {}

    raw_scores = np.array(list(scores.values()))
    exp_scores = np.exp(raw_scores - np.max(raw_scores))  # Stabilize for numerical precision
    softmax_scores = exp_scores / np.sum(exp_scores)
    return {key: softmax_score for key, softmax_score in zip(scores.keys(), softmax_scores)}

In [16]:
len(unique_words)

39796711

In [17]:
import math
import time
threshold=0.52
suggestion_threshold=0.007
dictionary={}
check=0
while True:
    sentence = input("")
    start=time.time()
    if sentence=='':
        break
    if(len(sentence.split(" "))>1):
        sentence=' '.join(sentence.split(" ")[-2:])
    output_words = []
    incorrect_sent=sentence.split(" ")
    typed_words=incorrect_sent
    output={}
    j=-1
    if(sentence[-1]==" "):
        predict(bi_trie, sentence, output, 1)
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        output = sort_acc_edit_distance(typed_words[j], output)
        break
    else:
        update_output(uni_trie, incorrect_sent[-1], incorrect_sent[-1], output, "uni")
        update_output(bi_trie, sentence, sentence, output, "bi")
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        output = sort_acc_edit_distance(typed_words[j], output)
        if typed_words[j] in words:    #6th oct changes : if typo is actually a valid word, do not autocorrect
            output_words.append(typed_words[j])
            break
        if(output==[]):
            break
        prob=output[0][1]
        if prob <threshold:
            prob=output[0][1]
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0][:len(typed_words[j])]) * (len(output[0][0]) / len(typed_words[j]))
            if len(typed_words[j]) >= 3 and len(typed_words[j]) <= 5:    
                if editdist <= 1:
                    prob += 0.2
            elif len(typed_words[j]) >= 6 and len(typed_words[j]) <= 10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
            elif len(typed_words[j]) >10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
                elif editdist <= 3:
                    prob += 0.1
        if prob >threshold:
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0]) * (len(output[0][0]) / len(typed_words[j]))
            if editdist>=len(output[0][0])/2.5:
                prob=prob-0.1
                if prob>threshold:
                    if typed_words[j] in output:
                        break
                    final_word=model_op_word
                else:
                    final_word=typed_words[j]
            else:
                final_word=model_op_word
            output_words.append(final_word)
        else:
            output_words.append(typed_words[j])
end=time.time()
print(end-start)
print("Suggestions:", [(k, v) for k, v in output if v > suggestion_threshold])
print("Inputs:", typed_words)
print("Outputs:", output_words)

h
h 
0.00019860267639160156
Suggestions: []
Inputs: ['h', '']
Outputs: []


In [18]:
uni_trie.calculate_size()

577377

In [20]:
bi_trie.calculate_size()

1955136

In [21]:
ride 

NameError: name 'ride' is not defined

In [22]:
words=[]
for i in data:
    i=i.split(" ")
    words.extend(i)
    words=list(set(words))
    if(len(words)>1000):
        break

import random
final=[]
for i in words:
    final.append(i+" ")
    length=random.randint(1,len(i))
    final.append(i[:length])
print(len(final))

NameError: name 'data' is not defined

In [25]:
import math
import time
threshold=0.52
suggestion_threshold=0.007
dictionary={}
check=0
start=time.time()
for i in tqdm(final[:200]):
    sentence = i    
    if sentence=='':
        continue
    if(len(sentence.split(" "))>1):
        sentence=' '.join(sentence.split(" ")[-2:])
    output_words = []
    incorrect_sent=sentence.split(" ")
    typed_words=incorrect_sent
    output={}
    j=-1
    if(sentence[-1]==" "):
        predict(bi_trie, sentence, output, 1)
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        z
        continue
    else:
        update_output(uni_trie, incorrect_sent[-1], incorrect_sent[-1], output, "uni")
        #update_output(bi_trie, sentence, sentence, output, "bi")
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        #output = sort_acc_edit_distance(typed_words[j], output)
        if typed_words[j] in words:    #6th oct changes : if typo is actually a valid word, do not autocorrect
            output_words.append(typed_words[j])
            continue
        if(output==[]):
            continue
        prob=output[0][1]
        continue
        if prob <threshold:
            prob=output[0][1]
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0][:len(typed_words[j])]) * (len(output[0][0]) / len(typed_words[j]))

            if len(typed_words[j]) >= 3 and len(typed_words[j]) <= 5:    
                if editdist <= 1:
                    prob += 0.2
            elif len(typed_words[j]) >= 6 and len(typed_words[j]) <= 10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
            elif len(typed_words[j]) >10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
                elif editdist <= 3:
                    prob += 0.1
        if prob >threshold:
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0]) * (len(output[0][0]) / len(typed_words[j]))
            if editdist>=len(output[0][0])/2.5:
                prob=prob-0.1
                if prob>threshold:
                    if typed_words[j] in output:
                        continue
                    final_word=model_op_word
                else:
                    final_word=typed_words[j]
            else:
                final_word=model_op_word
            output_words.append(final_word)
        else:
            output_words.append(typed_words[j])
#    print("Suggestions:", [(k, v) for k, v in output if v > suggestion_threshold])
#    print("Inputs:", typed_words)
#    print("Outputs:", output_words)
end=time.time()

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 234.90it/s]


In [26]:
(end-start)/200

0.004278112649917603

# Test

In [23]:
from glob import glob
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

from nltk import everygrams

from multiprocessing import Pool

df = pd.read_csv('../../../../Achyut_work/Trie_based_suggestions/resources/en-hi_normalised_unigrams.csv')
unigrams = list(df['0'].values)
unigrams=[i.lower() for i in unigrams]
wordlist=list(set(unigrams))
wordlist={i:None for i in wordlist}


In [25]:
days = sorted(glob("../../../../../../../NLP/CategoryPrediction/DataExtraction/june_2022/2022-06-04/*"))
print(len(days))
data=[]
import time
final={}
total_time=0
for csv in tqdm(days):
        #df=pd.read_csv(i)
        df = pd.read_csv(csv, on_bad_lines="skip", header=None, dtype={
            0: "str", 1: "str", 2: "str", 3: "str", 4: "str", 5: "str",
            6: "str", 7: "str", 8: "str", 9: "str", 10: "str"
        })
        #print(df[0].head(1))

        df.columns = ['text', 'package', 'language_code', 'timestamp', 'client_id', 'device', 'advertisement_id', 'latitude', 'longitude', 'country_code', 'region_code']
        df = df[['text', 'language_code']]
        df = df[df['language_code'].isin(['en', 'en-hi','en-ml','en-ta','en-te'])]
        df = df[df['text'].str.len() < 126]
        text=list(df["text"])
        text=[i.lower().strip() for i in df["text"]]
        text=list(set(text))
        data.extend(text)
        if(len(data)>50000):
            break
data=list(set(data))

527


  0%|                                                                                           | 0/527 [00:02<?, ?it/s]


In [26]:
len(data)

186219

In [27]:
sentences=[]
def are_all_words_in_list(sentence, word_list):
    # Convert the sentence into a list of words
    sentence_words = sentence.split()
    
    # Check if all words in the sentence are in the word_list
    return all(word in word_list for word in sentence_words)

# Example usage
for i in tqdm(data):
    if are_all_words_in_list(i, unique_words):
        sentences.append(i)

  0%|▎                                                                          | 730/186219 [02:35<10:57:19,  4.70it/s]


KeyboardInterrupt: 

In [ ]:
len(data)

In [ ]:
import time
def suggest(sentence):
    threshold=0.52
    suggestion_threshold=0.007
    dictionary={}
    check=0
    if sentence=='':
        return []
    if(len(sentence.split(" "))>1):
        sentence=' '.join(sentence.split(" ")[-2:])
    output_words = []
    incorrect_sent=sentence.split(" ")
    typed_words=incorrect_sent
    output={}
    j=-1
    if(sentence[-1]==" "):
        predict(bi_trie, sentence, output, 1)
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        output = sort_acc_edit_distance(typed_words[j], output)
        return output
    else:
        update_output(uni_trie, incorrect_sent[-1], incorrect_sent[-1], output, "uni")
        update_output(bi_trie, sentence, sentence, output, "bi")
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        output = sort_acc_edit_distance(typed_words[j], output)
        if typed_words[j] in words:    #6th oct changes : if typo is actually a valid word, do not autocorrect
            output_words.append(typed_words[j])
            return output
        if(output==[]):
            return output
        prob=output[0][1]
        if prob <threshold:
            prob=output[0][1]
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0][:len(typed_words[j])]) * (len(output[0][0]) / len(typed_words[j]))
            if len(typed_words[j]) >= 3 and len(typed_words[j]) <= 5:    
                if editdist <= 1:
                    prob += 0.2
            elif len(typed_words[j]) >= 6 and len(typed_words[j]) <= 10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
            elif len(typed_words[j]) >10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
                elif editdist <= 3:
                    prob += 0.1
        if prob >threshold:
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0]) * (len(output[0][0]) / len(typed_words[j]))
            if editdist>=len(output[0][0])/2.5:
                prob=prob-0.1
                if prob>threshold:
                    if typed_words[j] in output:
                        return output
                    final_word=model_op_word
                else:
                    final_word=typed_words[j]
            else:
                final_word=model_op_word
            output_words.append(final_word)
        else:
            output_words.append(typed_words[j])
    return([(k, v) for k, v in output if v > suggestion_threshold])

In [18]:
import math
d=[]
dd=[]
def get_tapSave_for_sentence(msg):
    tap_save = 0
    count=0
    words = msg.split()
    for j in range(len(words)):
        word = words[j]
        print("+"*50)
        check = True
        target_str1 = ''

        for i in range(len(word)):
            target_str1 += word[i]
            print("Looking word",word)
            print("Input",target_str1)

            if (i < (len(word) - 1)) and check:
                #ls = translate_beam_search(target_str, encoder_model, decoder_model, beam_width=20)
                ls=suggest(target_str1)
                ls = [word for word, _ in ls]
                print(ls)
                if word in ls:
                    print("found")
                    check = False
                    tap_save += (len(word) - i - 2)
                #else:
                #    dd.append([target_str,word,ls])
    return tap_save,count

In [21]:

ls =data[:10000]
total_tap = 0
tap_save = 0
i = 0
from tqdm import tqdm
total=[]
skip=[]
global tim
tim=0
for msg in tqdm(ls):
        space_count = msg.count(' ')
        total_tap += (len(msg))
        z=msg.split(" ")
        for i in z:
            if(i=="" or i==" "):
                continue
            if(i not in wordlist):
                try:
                    skip.append(msg)
                    i=1/0
                except:
                    pass
        t_save,t =  get_tapSave_for_sentence(msg)
        tap_save+=t_save
        total.append(t)
        print('Total tapSaved: ',(tap_save / total_tap)) 

  0%|                                                                                         | 0/10000 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word dikirim
Input d
['do', 'de', 'di']
Looking word dikirim
Input di
['di', 'din', 'diya']
Looking word dikirim
Input dik
['dik', 'dikh', 'dikha']
Looking word dikirim
Input diki
['diki', 'nhi', 'ni']
Looking word dikirim
Input dikir
['diki', 'diker']
Looking word dikirim
Input dikiri
['dikhai', 'dikhti', 'dikhi']
Looking word dikirim
Input dikirim
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word by
Input b


  0%|                                                                                 | 1/10000 [00:00<53:35,  3.11it/s]

['by', 'bs', 'be']
found
Looking word by
Input by
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word foto
Input f
['fr', 'fb', 'fir']
Looking word foto
Input fo
['fo', 'for', 'fon']
Looking word foto
Input fot
['fot', 'foto', 'fota']
found
Looking word foto
Input foto
Total tapSaved:  0.0
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word jp
Input j
['ji', 'jo', 'ja']
Looking word jp
Input jp
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word bhai
Input b
['by', 'bs', 'be']
Looking word bhai
Input bh
['bh', 'bhi', 'bhai']
found
Looking word bhai
Input bha
Looking word bhai
Input bhai
Total tapSaved:  0.045454545454545456


  0%|                                                                                 | 2/10000 [00:00<40:31,  4.11it/s]

...

  1%|▉                                                                              | 125/10000 [00:50<43:26,  3.79it/s]

['pr', 'pe', 'pn']
Looking word pron
Input pr
['pr', 'price', 'priya']
Looking word pron
Input pro
['pro', 'proof', 'proud']
Looking word pron
Input pron
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word zbardasti
Input z
['za', 'zal', 'zo']
Looking word zbardasti
Input zb
[]
Looking word zbardasti
Input zba
[]
Looking word zbardasti
Input zbar
[]
Looking word zbardasti
Input zbard
[]
Looking word zbardasti
Input zbarda
[]
Looking word zbardasti
Input zbardas
[]
Looking word zbardasti
Input zbardast
[]
Looking word zbardasti
Input zbardasti
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word kitchen
Input k
['ki', 'ka', 'kr']
Looking word kitchen
Input ki
['ki', 'kiya', 'kisi']
Looking word kitchen
Input kit
['kit', 'kiti', 'kita']
Looking word kitchen
Input kitc
['kiti', 'kita', 'kite']
Looking word kitchen
Input kitch
['kitchen', 'kith', 'kitchens']
found
Looking word kitchen
Input kitche
Looking word kitchen
Input kitchen
Total tapSaved:  0.0386608210442407

  1%|▉                                                                            | 126/10000 [00:51<1:07:12,  2.45it/s]

['to', 'tu', 'ta']
Looking word tere
Input te
['te', 'tere', 'teri']
found
Looking word tere
Input ter
Looking word tere
Input tere
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word taraha
Input t
['to', 'tu', 'ta']
Looking word taraha
Input ta
['ta', 'tab', 'tak']
Looking word taraha
Input tar
['tar', 'tara', 'tari']
Looking word taraha
Input tara
['tara', 'tarah', 'taraf']
Looking word taraha
Input tarah
['tarah', 'tarahi', 'tarahon']
Looking word taraha
Input taraha
Total tapSaved:  0.038884524744697564
++++++++++++++++++++++++++++++++++++++++++++++++++
Looking word banjara143kingkikk
Input b


KeyboardInterrupt: 

In [32]:
final

['song ',
 'so',
 'sunny ',
 's',
 'soch ',
 'soch',
 'normal ',
 'norma',
 'sochoo ',
 'so',
 'call ',
 'ca',
 'mm ',
 'mm',
 'erik ',
 'er',
 'ksm ',
 'ksm',
 'dp ',
 'dp',
 'jayega ',
 'jayeg',
 'holy ',
 'holy',
 'acara ',
 'ac',
 'potek ',
 'p',
 'pad ',
 'p',
 'zopte ',
 'zop',
 'raha ',
 'ra',
 'kn ',
 'kn',
 'ik ',
 'i',
 'raga ',
 'r',
 'too ',
 't',
 'heijauchi ',
 'heijauc',
 'kr ',
 'kr',
 'daal ',
 'daa',
 'videos ',
 'vide',
 'pathuttu ',
 'p',
 'kawrte ',
 'kawr',
 'nani ',
 'n',
 'right ',
 'rig',
 'ji ',
 'j',
 'gokul ',
 'gokul',
 'party ',
 'part',
 'avavolotha ',
 'avavolo',
 'imo ',
 'i',
 'reports ',
 'report',
 'lodallodalussun ',
 'lodal',
 'annam ',
 'an',
 'kahe ',
 'k',
 'gangama ',
 'gangama',
 'prnhu ',
 'prnh',
 'aawaz ',
 'aawaz',
 'solla ',
 'soll',
 'thak ',
 'thak',
 'sms ',
 's',
 'thee ',
 't',
 'soye ',
 's',
 'bc ',
 'bc',
 'yathuku ',
 'yat',
 'lga ',
 'lg',
 'din ',
 'din',
 'b ',
 'b',
 'pic ',
 'pi',
 'krna ',
 'k',
 'tesi ',
 'tesi',
 'korach 

In [30]:
words=[]
for i in data:
    i=i.split(" ")
    words.extend(i)
    words=list(set(words))
    if(len(words)>1000):
        break

import random
final=[]
for i in words:
    final.append(i+" ")
    length=random.randint(1,len(i))
    final.append(i[:length])
print(len(final))

2004


In [31]:
data

['date ka',
 'vituku poituya',
 'cek dta',
 'g ji',
 '9871086087 i love you',
 'nameplace anganea allam',
 'recharge kar de',
 'jai shree krishnan',
 'lal krishna',
 'pila eh gatag ko sa imo',
 'ksm h aapko',
 'nahi yaar ka ik baar liye bas',
 'avavolotha',
 'lodallodalussun',
 'mar bnya yg blng bgtu mem',
 'devaapa ala',
 'thought u at workat work',
 'genl ou renmen tout mizik mwn yo bb',
 'nee romba alagha irukanu',
 'subha 7 baje se riste daaro ke call suru ho jaate hai accha time paas ho jaayega',
 'kr naa',
 'haan thik hai',
 'yee sochoo',
 'bhjj de',
 'lavte balm aani zopte',
 'warna ini habis',
 'jo ab lagaege',
 'gpp bass',
 'kedarnath',
 'sansa inga',
 'je se katha mane padile kn nai kn heijauchi lo',
 'fon chak le',
 'aap mere pass hai',
 'omdash773',
 'kamte te iah show ng nong',
 'what ever u think',
 'anittmm',
 'ok bye kareja',
 'alledee',
 'ntr',
 'mene to ni bola roo',
 'jockey underwear female models hollywoodjockey',
 'yrr hume mera pic bekar lagta hai',
 'kha lo n pl

In [29]:
import math
import time
threshold=0.52
suggestion_threshold=0.007
dictionary={}
check=0
start=time.time()
for i in tqdm(final[:2000]):
    sentence = i    
    if sentence=='':
        continue
    if(len(sentence.split(" "))>1):
        sentence=' '.join(sentence.split(" ")[-2:])
    output_words = []
    incorrect_sent=sentence.split(" ")
    typed_words=incorrect_sent
    output={}
    j=-1
    if(sentence[-1]==" "):
        continue
        predict(bi_trie, sentence, output, 1)
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        #output = sort_acc_edit_distance(typed_words[j], output)
        continue
    else:
        check=uni_trie.starts_with(sentence)
        update_output(uni_trie, incorrect_sent[-1], incorrect_sent[-1], output, "uni")
        update_output(bi_trie, sentence, sentence, output, "bi")
        output=[(k, v) for k,v in output.items()]
        words=[x[0] for x in output]
        output = sorted(output, key=lambda x: x[1], reverse=True)[:3]
        #output = sort_acc_edit_distance(typed_words[j], output)
        if typed_words[j] in words:    #6th oct changes : if typo is actually a valid word, do not autocorrect
            output_words.append(typed_words[j])
            continue
        if(output==[]):
            continue
        prob=output[0][1]
        if prob <threshold:
            prob=output[0][1]
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0][:len(typed_words[j])]) * (len(output[0][0]) / len(typed_words[j]))
            if len(typed_words[j]) >= 3 and len(typed_words[j]) <= 5:    
                if editdist <= 1:
                    prob += 0.2
            elif len(typed_words[j]) >= 6 and len(typed_words[j]) <= 10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
            elif len(typed_words[j]) >10:
                if editdist <= 1:
                    prob += 0.2
                elif editdist <= 2:
                    prob += 0.15
                elif editdist <= 3:
                    prob += 0.1
        if prob >threshold:
            model_op_word=output[0][0]
            editdist=damerau_levenshtein_distance(typed_words[j],output[0][0]) * (len(output[0][0]) / len(typed_words[j]))
            if editdist>=len(output[0][0])/2.5:
                prob=prob-0.1
                if prob>threshold:
                    if typed_words[j] in output:
                        continue
                    final_word=model_op_word
                else:
                    final_word=typed_words[j]
            else:
                final_word=model_op_word
            output_words.append(final_word)
        else:
            output_words.append(typed_words[j])
#    print("Suggestions:", [(k, v) for k, v in output if v > suggestion_threshold])
#    print("Inputs:", typed_words)
#    print("Outputs:", output_words)
end=time.time()

TypeError: unhashable type: 'slice'

In [ ]:
(end-start)/2004*1000

In [31]:
(end-start)/2004*1000

9.72921477106517